
# Data from the Comète Film Festival

## Importing the necessary libraries and files

In [1]:
import pandas as pd
import math

# Import the Altair library
# by convention, we use alt to save typing, but you can just 'import altair' if you wish
import altair as alt
from altair import datum
import os as os

alt.data_transformers.enable('default',max_rows=None) # work-around to let Altair handle larger data sets

DataTransformerRegistry.enable('default')

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Compétition-générale-Notation.csv to Compétition-générale-Notation.csv


## Modifying the data to clean it up before using it

In [3]:
df = pd.read_csv('Compétition-générale-Notation.csv', sep=';')

In [4]:
df.iloc[574,:]

 Nom du film                                       Fabriquer la guitare de mes rêves
Nom du noteur                                                                  Mehdi
Synopsis /10                                                                       7
Son /10                                                                            7
Décors et Costumes /10                                                             6
Qualité tournage /10                                                               6
Qualité montage /10                                                                7
Jeu d'acteurs /10                                                                  6
Bonus /2 (quali, originalité)                                                    NaN
TOTAL /12                                                                        6,5
En langue étrangère (oui / non)                                                  non
Commentaire                        L'idee est pas mal mais la rea

In [5]:
df.columns

Index([' Nom du film', 'Nom du noteur', 'Synopsis /10', 'Son /10',
       'Décors et Costumes /10', 'Qualité tournage /10', 'Qualité montage /10',
       'Jeu d'acteurs /10', 'Bonus /2 (quali, originalité)', 'TOTAL /12',
       'En langue étrangère (oui / non)', 'Commentaire', 'Unnamed: 12',
       'Unnamed: 13'],
      dtype='object')

In [6]:
del df['Unnamed: 12']
del df['Unnamed: 13']

In [7]:
df = df.rename(columns={' Nom du film' : 'Nom du film', 'En langue étrangère (oui / non)': 'Langue étrangère (oui/non)','Jeu d\'acteurs /10':'Jeu des acteurs /10'})

In [8]:
def process_graders(df_row) :
  df_row['Nom du noteur'] = df_row['Nom du noteur'].lower()
  if df_row['Nom du noteur'] == 'alex ' : df_row['Nom du noteur'] = 'alex'
  if df_row['Nom du noteur'] == 'alexou' : df_row['Nom du noteur'] = 'alex'
  if df_row['Nom du noteur'] == 'nico /alex' : df_row['Nom du noteur'] = 'nicolas'
  if df_row['Nom du noteur'] == 'nico ' : df_row['Nom du noteur'] = 'nicolas'
  if df_row['Nom du noteur'] == 'nico' : df_row['Nom du noteur'] = 'nicolas'
  if df_row['Nom du noteur'] == 'nicolas ' : df_row['Nom du noteur'] = 'nicolas'
  if df_row['Nom du noteur'] == 'jérémie ' : df_row['Nom du noteur'] = 'jérémie'
  if df_row['Nom du noteur'] == 'lucie' : df_row['Nom du noteur'] = 'clément'
  if df_row['Nom du noteur'] == 'joanne ' : df_row['Nom du noteur'] = 'joanne'
  if df_row['Nom du noteur'] == 'tiff' : df_row['Nom du noteur'] = 'tiffany'
  if df_row['Nom du noteur'] == 'gregoire' : df_row['Nom du noteur'] = 'grégoire'

In [9]:
df.apply(process_graders, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
587    None
588    None
589    None
590    None
591    None
Length: 592, dtype: object

In [10]:
noteurs = []
for elt in df['Nom du noteur'] :
  if elt not in noteurs : noteurs.append(elt)
noteurs

['arthur',
 'grégoire',
 'joanne',
 'eliott',
 'marine',
 'alex',
 'jérémie',
 'théophile',
 'tiffany',
 'pauline',
 'laurent',
 'estelle',
 'mehdi',
 'aurèle',
 'nicolas',
 'clément',
 'bastien',
 'matthieu']

In [11]:
df = df.fillna(0)
df

,Nom du film,Nom du noteur,Synopsis /10,Son /10,Décors et Costumes /10,Qualité tournage /10,Qualité montage /10,Jeu des acteurs /10,"Bonus /2 (quali, originalité)",TOTAL /12,Langue étrangère (oui/non),Commentaire
0,Chaque seconde compte,arthur,6,6,8,7,7,"7,5",1,"7,916666667",non,Le concept est super sympa. Vraiment réaliste ...
1,Chaque seconde compte,grégoire,"6,5",8,7,7,8,6,0,"7,083333333",non,"tout n'est pas très clair, difficile à suivre"
2,Chaque seconde compte,joanne,"5,5",4,5,5,"6,5",6,1,"6,333333333",non,projet expérimentalement intéréssant mais il n...
3,Premier Rôle,eliott,9,7,7,8,8,9,0,8,non,Franchement c'est quali. l'histoire est cool m...
4,Premier Rôle,joanne,6,7,8,8,7,5,0,"6,833333333",non,C'est du dejà vu mais ce n'est pas mal réalisé...
...,...,...,...,...,...,...,...,...,...,...,...,...
587,Retour aux cuves,matthieu,"7,5","6,5",3,4,3,5,0,"4,833333333",non,"Pauvre en idées visuelles, mais le thème est i..."
588,Attends-moi je meurs,matthieu,7,7,9,9,7,6,"0,5",8,non,"Travail d'animation très qualitatif, beau jeu ..."
589,Attends-moi je meurs,pauline,"6,5","8,5","8,5",9,7,6,0,"7,583333333",non,Beaucoup de travail dans l'animation des poupé...
590,BAD BITCHES (MAUVAISES CHIENNES),joanne,8,8,7,7,7,9,"0,5","8,166666667",non,court métrage dont le thème est original. Le t...


In [12]:
# New dataframe with cleaned grades
df_cleaned_grades = pd.DataFrame(columns=df.columns)

score_cols = [
    'Synopsis /10',
    'Son /10',
    'Décors et Costumes /10',
    'Qualité tournage /10',
    'Qualité montage /10',
    'Jeu des acteurs /10',
    'Bonus /2 (quali, originalité)',
    'TOTAL /12',
]

for entry in df.index:
    movie_row = {col: round(float((str(df[col][entry]).replace(',', '.'))), 2) for col in score_cols}
    movie_row['Bonus /12 (quali, originalité)'] = round(movie_row['Bonus /2 (quali, originalité)'] * 6, 2)
    movie_row['Nom du film'] = df['Nom du film'][entry]
    movie_row['Nom du noteur'] = df['Nom du noteur'][entry]
    movie_row['Langue étrangère (oui/non)'] = df['Langue étrangère (oui/non)'][entry] == 'oui'
    movie_row['Commentaire'] = df['Commentaire'][entry]
    df_cleaned_grades = df_cleaned_grades.append(movie_row, ignore_index=True)

score_cols += ['Bonus /12 (quali, originalité)']

df_cleaned_grades.sort_values('Nom du film')

,Nom du film,Nom du noteur,Synopsis /10,Son /10,Décors et Costumes /10,Qualité tournage /10,Qualité montage /10,Jeu des acteurs /10,"Bonus /2 (quali, originalité)",TOTAL /12,Langue étrangère (oui/non),Commentaire,"Bonus /12 (quali, originalité)"
49,silence (sous-titré),grégoire,7.0,8.0,6.0,6.0,6.0,6.0,0.5,7.00,True,l'idée est vraiment bonne et l'ambiance sonore...,3.0
50,silence (sous-titré),théophile,7.0,6.0,6.0,6.0,6.5,6.0,0.5,6.75,True,Léger problème sur la qualité sonore des plans...,3.0
92,(Une parenthèse),marine,4.0,6.0,7.0,5.0,5.0,6.0,0.0,5.50,False,Je crois que ce court métrage est un peu trop ...,0.0
91,(Une parenthèse),arthur,5.0,6.5,6.0,6.5,5.0,6.0,0.0,5.83,False,J'ai passé 20 min assez mal à l'aise. Y a un c...,0.0
81,12 Renan,théophile,5.0,6.0,6.0,6.0,6.0,4.0,0.0,5.50,False,C'est un documentaire et puis les gros plans s...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,À la croisée des destins,aurèle,6.0,7.0,8.0,9.0,9.0,5.5,0.0,7.42,False,Bien tourné et les effets spéciaux sont très b...,0.0
117,À la croisée des destins,marine,7.0,8.5,8.0,7.5,9.0,7.0,0.5,8.33,False,Les plans et la colorimétrie sont vraiment bie...,3.0
317,Électrique,joanne,7.5,7.0,7.0,8.0,7.5,7.0,0.5,7.83,False,Film efficace mais j'aurais aimer en voir plus...,3.0
318,Électrique,aurèle,8.0,7.0,7.0,8.0,7.5,7.0,0.5,7.92,False,Un film bien réalisé mais il manque une pointe...,3.0


In [13]:
#New dataframe where there is only one entry per film, the grades are the average of the grades
df_average=pd.DataFrame(columns=df_cleaned_grades.columns)
del df_average['Nom du noteur']
del df_average['Commentaire']

#First, we find every lines for each movie and we store them in a dictionnary
movies_dict = {}
for i in df_cleaned_grades.index:
    title = df_cleaned_grades['Nom du film'][i]
    movies_dict[title] = movies_dict[title] + [i] if title in movies_dict.keys() else [i]

for movie in movies_dict.keys():
    count = len(movies_dict[movie])
    avg_row = {col: round(sum(df_cleaned_grades[col][entry] for entry in movies_dict[movie]) / count, 2) for col in score_cols}      
    avg_row['Nom du film'] = movie
    avg_row['Langue étrangère (oui/non)'] = df_cleaned_grades['Langue étrangère (oui/non)'][movies_dict[movie][0]]
    df_average = df_average.append(avg_row, ignore_index=True)
    
df_average.sort_values('TOTAL /12')

,Nom du film,Synopsis /10,Son /10,Décors et Costumes /10,Qualité tournage /10,Qualité montage /10,Jeu des acteurs /10,"Bonus /2 (quali, originalité)",TOTAL /12,Langue étrangère (oui/non),"Bonus /12 (quali, originalité)"
84,Ma fusée,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,False,0.0
294,Chi Mouja,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,False,0.0
37,Déclaration d'amour hors-pair,1.5,0.00,0.00,1.0,1.0,1.00,0.00,0.75,False,0.0
208,La Maison près du lac,1.0,2.50,2.50,2.5,3.0,1.00,0.00,2.08,False,0.0
60,DONNE TA LANGUE AU...,3.0,1.00,4.50,3.0,2.5,2.00,0.00,2.67,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...
225,Les premiers hommes dansaient,9.0,8.50,8.50,9.5,8.5,7.75,1.00,9.62,False,6.0
312,Las Nadie,8.0,9.00,8.50,9.0,8.0,9.50,1.00,9.67,True,6.0
262,Le petit sapin,9.0,6.00,10.00,9.0,10.0,8.50,1.00,9.75,False,6.0
264,Bloody Mary,9.0,9.25,8.75,9.5,9.0,10.00,0.75,10.00,True,4.5


## Using the dataframes


### Common data for plots

In [14]:
# fields definitions
grades = 'Grades'
synopsis = 'Synopsis /10'
sound = 'Son /10'
costumes = 'Décors et Costumes /10'
tournage = 'Qualité tournage /10'
montage = 'Qualité montage /10'
acteurs = 'Jeu des acteurs /10'
bonus = 'Bonus /12 (quali, originalité)'
total = 'TOTAL /12'
foreign_language = 'Langue étrangère (oui/non)'

fields = [synopsis,sound,costumes,tournage,montage,acteurs,bonus]
fields_for_tooltip = ['Nom du film', foreign_language] + fields + [total]

# properties
height = 200
bar_width = 8
step_width = 9

coloring = alt.Color('Field:N', legend=alt.Legend(orient="top", title=None, values=fields))
field_sortation = alt.SortArray(fields)

# selectors
multi_selection = alt.selection_multi(fields=['Nom du film'])
interval_selection = alt.selection_interval(
    encodings = ['x'],
    on="[mousedown[event.altKey], mouseup] > mousemove",
    mark=alt.BrushConfig(fill="#fdbb84", fillOpacity=0.5, stroke="#e34a33")
)
film_selectors = multi_selection | interval_selection

# dropdowns and sliders
noteur_dropdown = alt.binding_select(options=noteurs)
noteur_selection = alt.selection_single(fields=['Nom du noteur'], bind=noteur_dropdown, name='Grader', init={'Nom du noteur': 'arthur'})
language_dropdown = alt.binding_select(options=[None, True, False])
language_selection = alt.selection_single(fields=[foreign_language], bind=language_dropdown, name='Foreign language')
min_grade_slider = alt.binding_range(min=0, max=10, step=0.5, name='min. Grade:')
min_grade_selector = alt.selection_single(name="MinGrade", fields=['min_grade'], bind=min_grade_slider, init={'min_grade': 0})

In [15]:
def create_chart(dataframe, sort_selector, selection=False, selector=None):
  if (sort_selector=="grades"):
      field = grades
      fields = [synopsis,sound,costumes,tournage,montage,acteurs,bonus]
  elif (sort_selector=="synopsis"):
      field = synopsis
      fields = [synopsis,sound,costumes,tournage,montage,acteurs,bonus]
  elif (sort_selector=="sound"):
      field = sound
      fields = [sound,synopsis,costumes,tournage,montage,acteurs,bonus]
  elif (sort_selector=="costumes"):
      field = costumes
      fields = [costumes,sound,synopsis,tournage,montage,acteurs,bonus]
  elif (sort_selector=="tournage"):
      field = tournage
      fields = [tournage,costumes,sound,synopsis,montage,acteurs,bonus]
  elif (sort_selector=="montage"):
      field = montage
      fields = [montage,tournage,costumes,sound,synopsis,acteurs,bonus]
  elif (sort_selector=="acteurs"):
      field = acteurs
      fields = [acteurs,montage,tournage,costumes,sound,synopsis,bonus]
  elif (sort_selector=="bonus"):
      field = bonus
      fields = [bonus,acteurs,montage,tournage,costumes,sound,synopsis]

  base = alt.Chart(dataframe).transform_fold(fields, as_=['Field', 'Grades']).mark_bar(size=bar_width).encode(
        x=alt.X('Nom du film:N', axis=alt.Axis(labelLimit=100, labelAngle=90), sort=alt.EncodingSortField(field=field, op='average', order='descending')),
        y=alt.Y('Grades:Q', scale=alt.Scale(domain=(0, 60))),
        color=alt.condition(selector, coloring, alt.value('lightgrey')),
        tooltip=fields_for_tooltip,
      ).transform_filter(
          language_selection
      ).transform_filter(
          datum[total] > min_grade_selector['min_grade']
      ).properties(height=height, width=alt.Step(step_width))
  return base if sort_selector=="grades" else base.encode(order=alt.Order(aggregate="mean", field=field, type="quantitative"))

In [16]:
print("The options for sort_by are :\n")
print("'grades','synopsis','sound','costumes','tournage','montage','acteurs','bonus'")
sort_by = input("Which option do you want to sort by ? ")

bar_chart = create_chart(df_average, sort_by, True, film_selectors).add_selection(multi_selection, interval_selection)

total_chart = alt.Chart(df_average).mark_bar(size=bar_width, fill='#000').encode(
  x=alt.X('Nom du film:N', axis=None, sort=alt.EncodingSortField(field="TOTAL /12", op='average', order='descending')),
  y=alt.Y('TOTAL /12:Q'),
  color=alt.condition(film_selectors, 'Field:N', alt.value('lightgrey'))
).add_selection(multi_selection, interval_selection).properties(height=height, width=alt.Step(step_width),title="Film List")

text = total_chart.mark_text(
    clip=True,
    orient='vertical',
    angle=90,
    align='left',
    baseline='middle'
).encode(
    color=alt.condition(film_selectors, alt.value('#111'), alt.value('lightgrey')),
    text='Nom du film:N',
    tooltip=fields_for_tooltip
)

The options for sort_by are :

'grades','synopsis','sound','costumes','tournage','montage','acteurs','bonus'
Which option do you want to sort by ? grades



#JURY VIEW

In [21]:
bar =alt.Chart(df).mark_circle(size=80).encode(
    x=alt.X('Nom du noteur:N'),
    y=alt.Y('TOTAL /12:Q')
).properties(width=800,height=500,)

line = alt.Chart(df).mark_tick(color='red',size=30, thickness=2).encode(
    x=alt.X('Nom du noteur:N'),
    y='mean(TOTAL /12):Q'
)

In [17]:
#grades per category per person
# notes_par_categorie = alt.Chart(df_cleaned_grades).mark_circle(size=100).transform_fold(
#           [acteurs,montage,tournage,costumes,sound,synopsis],
#           as_=['Field', 'Grade']
#           ).encode(
#     x='Field:N',
#     y=alt.Y('Grade:Q', sort='-y'),
#     color=coloring
# ).transform_filter(
#     interval
# ).transform_filter(
#     noteur_selection
# ).properties(width=200,height=height,selection=interval)


fields_for_plot = fields.copy()
fields_for_plot.remove(bonus)
# box_plot = alt.Chart(df_cleaned_grades).mark_boxplot().transform_fold(
#       fields_for_plot,
#       as_=['Field', 'Grades']
#     ).encode(
#       x=alt.X('Field:N', axis=None, sort=field_sortation),
#       y=alt.Y('Grades:Q', axis=alt.Axis(title='Grades Distribution'), scale=alt.Scale(domain=(0, 10))),
#       color=coloring
# ).transform_filter(
#     noteur_selection
# ).properties(height=200)

strip_plot = alt.Chart(df_cleaned_grades, width=50).mark_circle().transform_fold(
      fields_for_plot,
      as_=['Field', 'Grades']
    ).encode(
    size=alt.condition(film_selectors, alt.value(30), alt.value(5)),
    opacity=alt.condition(film_selectors, alt.value(1), alt.value(.3)),
    x=alt.X('jitter:Q', title=None, axis=None, scale=alt.Scale(domain=(-2,2), clamp=True)),
    y=alt.Y('Grades:Q', axis=alt.Axis(title='Grades Distribution'), scale=alt.Scale(domain=(0, 10))),
    color=alt.condition(film_selectors, coloring, alt.value('grey')),
    column=alt.Column('Field:N', header=alt.Header(labels=False), title=None, spacing=0, sort=field_sortation),
    tooltip=fields_for_tooltip
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
).transform_filter(
    noteur_selection
).transform_filter(
    language_selection
).transform_filter(
    datum[total] > min_grade_selector['min_grade']
).properties(height=200).interactive().add_selection(multi_selection)

#Average of the grades per category
# line = alt.Chart(df_cleaned_grades).mark_tick(color='black',size=30, thickness=2).transform_fold(
#           [acteurs,montage,tournage,costumes,sound,synopsis],
#           as_=['Field', 'Grade']
#           ).encode(
#     x=alt.X('Field:N', axis=None),
#     y=alt.Y('mean(Grade):Q', scale=alt.Scale(domain=(0, 10)))
# ).transform_filter(interval)

#Total grade given to movies by a grader
text_noteur = alt.Chart(df_cleaned_grades).mark_text(
    clip=True,
    orient='vertical',
    angle=90,
    align='left',
    baseline='middle'
).encode(
    x=alt.X('Nom du film:N', axis=None, sort=alt.EncodingSortField(field=total, op='average', order='descending')),
    y=alt.Y('mean(TOTAL /12):Q', axis=alt.Axis(title='TOTAL /12 for Grader'), scale=alt.Scale(domain=(0, 10))),
    color=alt.condition(film_selectors, alt.value('#111'), alt.value('lightgrey')),
    text='Nom du film:N',
    tooltip=fields_for_tooltip + ['Commentaire:N']
).transform_filter(
    noteur_selection
).transform_filter(
    language_selection
).transform_filter(
    datum[total] > min_grade_selector['min_grade']
).add_selection(
    multi_selection, interval_selection, noteur_selection, language_selection, min_grade_selector
).properties(height=200, width=alt.Step(step_width))

# bar_with_noteurs = alt.Chart(df_cleaned_grades).transform_fold(fields, as_=['Field', 'Grades']).mark_bar(size=bar_width).encode(
#     x=alt.X('Nom du film:N', axis=alt.Axis(labelLimit=100, labelAngle=90), sort=alt.EncodingSortField(field='Grades', op='average', order='descending')),
#     y=alt.Y('average(Grades):Q', scale=alt.Scale(domain=(0, 65))),
#     color=alt.condition(film_selectors, coloring, alt.value('lightgrey')),
#     tooltip=['Nom du film'] + fields + [total],
# ).properties(height=200, width=alt.Step(step_width))

## Film View

In [18]:
film_details = alt.Chart(df_cleaned_grades).mark_bar().transform_fold(
      fields_for_plot,
      as_=['Field', 'Grades']
    ).encode(
      x=alt.X('Field:N', axis=None, sort=field_sortation),
      y=alt.Y('mean(Grades):Q', scale=alt.Scale(domain=(0, 10))),
      color=coloring,
      column=alt.Column('Nom du noteur:N', title='Grader')
).transform_filter(
    film_selectors
).transform_filter(
    language_selection
).transform_filter(
    datum[total] > min_grade_selector['min_grade']
).properties(height=200, width=alt.Step(step_width), title="Film details")

## Complete Chart

In [19]:
complete_chart = bar_chart & film_details & (strip_plot | text_noteur)
complete_chart = complete_chart.configure(background='#fef9f2').configure_axis(labelFontSize=11,titleFontSize=16).configure_legend(labelFontSize=16,titleFontSize=20,labelLimit=500, orient='bottom').configure_title(fontSize=30, anchor='middle')
complete_chart

alt.VConcatChart(...)

In [29]:
complete_chart.save('index.html')